# Data Collection

- Songs lyrics corpus

The data used in this project is taken from ["Spotify Million Playlist Dataset Challenge"](https://www.aicrowd.com/challenges/spotify-million-playlist-dataset-challenge) - a continuation of a data science research challenge focused on music recommendation organized by Spotify (See [RecSys Challenge 2018](http://www.recsyschallenge.com/2018/)).

Another music recommendation challenge that we've considered to base our work on is <https://www.kaggle.com/c/msdchallenge/overview>. However, due to its old age (2012), smaller scale and rigit data formats, the former dataset was preferred.

Our data consists of:

1. From <https://www.aicrowd.com/challenges/spotify-million-playlist-dataset-challenge/dataset_files>:

spotify_million_playlist_dataset/ - 1 million playlists consisting of over 2 million unique tracks by nearly 300,000 artists. Created by US Spotify users between January 2010 and November 2017. **Used for both the training set and the test set**


2. all_songs.json - all unique songs generated from the playlists dataset

In [4]:
import pandas as pd

# Demonstration of the file's format
all_songs_df = pd.read_json('./data/all_songs.json').T
all_songs_df.head()

3. Retrieved from <https://api.spotify.com/>:

audio_features.json - Various audio features collection we created out of the songs from the playlists dataset. Retrieved from Spotify public API.

4. Retrieved from <https://genius.com/>:

lyrics_corpus.json - Lyrics collection of many of the songs from the playlists dataset. Scraped from Genius Lyrics site and public API.